In [1]:
import requests
from bs4 import BeautifulSoup as bs
import json
import re
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# used the webpage of popular YouTubeCody Ko
response = requests.get("https://www.youtube.com/c/CodyKo/videos")
response

<Response [200]>

In [3]:
soup = bs(response.text, 'html.parser')

In [4]:
# initial inspection of html script
print(soup.prettify())

<!DOCTYPE html>
<html lang="en" style="font-size: 10px;font-family: Roboto, Arial, sans-serif;" system-icons="" typography="" typography-spacing="">
 <head>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="AsKJ0YJfrprDHN9cO+Arkwlyt0SqGVbRekp6Ho6B0zL8/aWcEEa7mYjLrXVRY4E3+ZPTupaMmCxJOtl4ava9CgAAAABteyJvcmlnaW4iOiJodHRwczovL3lvdXR1YmUuY29tOjQ0MyIsImZlYXR1cmUiOiJQcml2YWN5U2FuZGJveEFkc0FQSXMiLCJleHBpcnkiOjE2Njk3NjYzOTksImlzU3ViZG9tYWluIjp0cnVlfQ==" http-equiv="origin-trial"/>
  <script nonce="aHjBc4mUVkC5CSrb5wcucg">
   var ytcfg={d:function(){return window.yt&&yt.config_||ytcfg.data_||(ytcfg.data_={})},get:function(k,o){return k in ytcfg.d()?ytcfg.d()[k]:o},set:function(){var a=arguments;if(a.length>1)ytcfg.d()[a[0]]=a[1];else for(var k in a[0])ytcfg.d()[k]=a[0][k]}};
window.ytcfg.set('EMERGENCY_BASE_URL', '\/error_204?t\x3djserror\x26level\x3dERROR\x26client.name\x3d1\x26client.version\x3d2.20220829.00.00');
  </script>
  <script nonce="aHjBc4mUVkC5CSrb5wcucg">
  

In [5]:
# returns list of script tags, one of which holds the title of the videos
results = soup.find_all('script')
results

[<script nonce="aHjBc4mUVkC5CSrb5wcucg">var ytcfg={d:function(){return window.yt&&yt.config_||ytcfg.data_||(ytcfg.data_={})},get:function(k,o){return k in ytcfg.d()?ytcfg.d()[k]:o},set:function(){var a=arguments;if(a.length>1)ytcfg.d()[a[0]]=a[1];else for(var k in a[0])ytcfg.d()[k]=a[0][k]}};
 window.ytcfg.set('EMERGENCY_BASE_URL', '\/error_204?t\x3djserror\x26level\x3dERROR\x26client.name\x3d1\x26client.version\x3d2.20220829.00.00');</script>,
 <script nonce="aHjBc4mUVkC5CSrb5wcucg">(function(){window.yterr=window.yterr||true;window.unhandledErrorMessages={};window.unhandledErrorCount=0;
 window.onerror=function(msg,url,line,columnNumber,error){var err;if(error)err=error;else{err=new Error;err.stack="";err.message=msg;err.fileName=url;err.lineNumber=line;if(!isNaN(columnNumber))err["columnNumber"]=columnNumber}var message=String(err.message);if(!err.message||message in window.unhandledErrorMessages||window.unhandledErrorCount>=5)return;window.unhandledErrorCount+=1;window.unhandledErr

In [6]:
# search each tag for the video title
index = 0
for result in results:
    print(index, result)
    print("__________________")
    index += 1

0 <script nonce="aHjBc4mUVkC5CSrb5wcucg">var ytcfg={d:function(){return window.yt&&yt.config_||ytcfg.data_||(ytcfg.data_={})},get:function(k,o){return k in ytcfg.d()?ytcfg.d()[k]:o},set:function(){var a=arguments;if(a.length>1)ytcfg.d()[a[0]]=a[1];else for(var k in a[0])ytcfg.d()[k]=a[0][k]}};
window.ytcfg.set('EMERGENCY_BASE_URL', '\/error_204?t\x3djserror\x26level\x3dERROR\x26client.name\x3d1\x26client.version\x3d2.20220829.00.00');</script>
__________________
1 <script nonce="aHjBc4mUVkC5CSrb5wcucg">(function(){window.yterr=window.yterr||true;window.unhandledErrorMessages={};window.unhandledErrorCount=0;
window.onerror=function(msg,url,line,columnNumber,error){var err;if(error)err=error;else{err=new Error;err.stack="";err.message=msg;err.fileName=url;err.lineNumber=line;if(!isNaN(columnNumber))err["columnNumber"]=columnNumber}var message=String(err.message);if(!err.message||message in window.unhandledErrorMessages||window.unhandledErrorCount>=5)return;window.unhandledErrorCount+=1;w

In [7]:
# element 33 holds the script tag with the title information
result_with_title = results[33]
result_with_title

<script nonce="aHjBc4mUVkC5CSrb5wcucg">var ytInitialData = {"responseContext":{"serviceTrackingParams":[{"service":"GFEEDBACK","params":[{"key":"route","value":"channel.videos"},{"key":"is_casual","value":"false"},{"key":"is_owner","value":"false"},{"key":"is_monetization_enabled","value":"true"},{"key":"num_shelves","value":"6+"},{"key":"is_alc_surface","value":"false"},{"key":"browse_id","value":"UCfp86n--4JvqKbunwSI2lYQ"},{"key":"logged_in","value":"0"},{"key":"e","value":"24216097,1714259,24264860,24267564,23744176,24245609,23940247,24218924,23986029,24114244,24260783,39321934,23998056,24262346,24169501,24113426,24007246,24004644,24269956,24077241,24166867,24211178,24181174,23882502,24034168,23966208,24135310,24152443,24229161,24264385,23946420,24263272,24197275,24261927,23934970,23804281,24241661,24246430,23983296,24253956,24248232,24255543,24140247,24227844,24275320,24199724,24272071,24226335,24270273,24191629,24037231,24161116,24269016,24256690,9406121,24215196,24264314,24248385

In [8]:
type(result_with_title)

bs4.element.Tag

In [9]:
result_with_title.name

'script'

In [10]:
result_with_title.attrs

{'nonce': 'aHjBc4mUVkC5CSrb5wcucg'}

In [11]:
content = result_with_title.contents

In [12]:
type(content)

list

In [13]:
len(content)

1

In [14]:
content

['var ytInitialData = {"responseContext":{"serviceTrackingParams":[{"service":"GFEEDBACK","params":[{"key":"route","value":"channel.videos"},{"key":"is_casual","value":"false"},{"key":"is_owner","value":"false"},{"key":"is_monetization_enabled","value":"true"},{"key":"num_shelves","value":"6+"},{"key":"is_alc_surface","value":"false"},{"key":"browse_id","value":"UCfp86n--4JvqKbunwSI2lYQ"},{"key":"logged_in","value":"0"},{"key":"e","value":"24216097,1714259,24264860,24267564,23744176,24245609,23940247,24218924,23986029,24114244,24260783,39321934,23998056,24262346,24169501,24113426,24007246,24004644,24269956,24077241,24166867,24211178,24181174,23882502,24034168,23966208,24135310,24152443,24229161,24264385,23946420,24263272,24197275,24261927,23934970,23804281,24241661,24246430,23983296,24253956,24248232,24255543,24140247,24227844,24275320,24199724,24272071,24226335,24270273,24191629,24037231,24161116,24269016,24256690,9406121,24215196,24264314,24248385,24187043,24239634,24260441,24185614,

In [15]:
content[0]

'var ytInitialData = {"responseContext":{"serviceTrackingParams":[{"service":"GFEEDBACK","params":[{"key":"route","value":"channel.videos"},{"key":"is_casual","value":"false"},{"key":"is_owner","value":"false"},{"key":"is_monetization_enabled","value":"true"},{"key":"num_shelves","value":"6+"},{"key":"is_alc_surface","value":"false"},{"key":"browse_id","value":"UCfp86n--4JvqKbunwSI2lYQ"},{"key":"logged_in","value":"0"},{"key":"e","value":"24216097,1714259,24264860,24267564,23744176,24245609,23940247,24218924,23986029,24114244,24260783,39321934,23998056,24262346,24169501,24113426,24007246,24004644,24269956,24077241,24166867,24211178,24181174,23882502,24034168,23966208,24135310,24152443,24229161,24264385,23946420,24263272,24197275,24261927,23934970,23804281,24241661,24246430,23983296,24253956,24248232,24255543,24140247,24227844,24275320,24199724,24272071,24226335,24270273,24191629,24037231,24161116,24269016,24256690,9406121,24215196,24264314,24248385,24187043,24239634,24260441,24185614,2

In [16]:
str(content[0])

'var ytInitialData = {"responseContext":{"serviceTrackingParams":[{"service":"GFEEDBACK","params":[{"key":"route","value":"channel.videos"},{"key":"is_casual","value":"false"},{"key":"is_owner","value":"false"},{"key":"is_monetization_enabled","value":"true"},{"key":"num_shelves","value":"6+"},{"key":"is_alc_surface","value":"false"},{"key":"browse_id","value":"UCfp86n--4JvqKbunwSI2lYQ"},{"key":"logged_in","value":"0"},{"key":"e","value":"24216097,1714259,24264860,24267564,23744176,24245609,23940247,24218924,23986029,24114244,24260783,39321934,23998056,24262346,24169501,24113426,24007246,24004644,24269956,24077241,24166867,24211178,24181174,23882502,24034168,23966208,24135310,24152443,24229161,24264385,23946420,24263272,24197275,24261927,23934970,23804281,24241661,24246430,23983296,24253956,24248232,24255543,24140247,24227844,24275320,24199724,24272071,24226335,24270273,24191629,24037231,24161116,24269016,24256690,9406121,24215196,24264314,24248385,24187043,24239634,24260441,24185614,2

In [17]:
# convert javascript code to string
data = str(content[0])
data

'var ytInitialData = {"responseContext":{"serviceTrackingParams":[{"service":"GFEEDBACK","params":[{"key":"route","value":"channel.videos"},{"key":"is_casual","value":"false"},{"key":"is_owner","value":"false"},{"key":"is_monetization_enabled","value":"true"},{"key":"num_shelves","value":"6+"},{"key":"is_alc_surface","value":"false"},{"key":"browse_id","value":"UCfp86n--4JvqKbunwSI2lYQ"},{"key":"logged_in","value":"0"},{"key":"e","value":"24216097,1714259,24264860,24267564,23744176,24245609,23940247,24218924,23986029,24114244,24260783,39321934,23998056,24262346,24169501,24113426,24007246,24004644,24269956,24077241,24166867,24211178,24181174,23882502,24034168,23966208,24135310,24152443,24229161,24264385,23946420,24263272,24197275,24261927,23934970,23804281,24241661,24246430,23983296,24253956,24248232,24255543,24140247,24227844,24275320,24199724,24272071,24226335,24270273,24191629,24037231,24161116,24269016,24256690,9406121,24215196,24264314,24248385,24187043,24239634,24260441,24185614,2

In [41]:
# use regular expressions to locate the titles
final_results = re.findall("\"title\":{\"runs\":.{0,1200},\"accessibility", data)
final_results

['"title":{"runs":[{"text":"I ran an ULTRA marathon"}],"accessibility',
 '"title":{"runs":[{"text":"THE. BUTTON."}],"accessibility',
 '"title":{"runs":[{"text":"Bridalplasty - what the hell was this show?"}],"accessibility',
 '"title":{"runs":[{"text":"Roasting Your Fridge"}],"accessibility',
 '"title":{"runs":[{"text":"The Button (This Time It\'s PERSONAL)"}],"accessibility',
 '"title":{"runs":[{"text":"Tinder Nightmares 4"}],"accessibility',
 '"title":{"runs":[{"text":"The Button Gets Crazy"}],"accessibility',
 '"title":{"runs":[{"text":"The Button is BACK!!"}],"accessibility',
 '"title":{"runs":[{"text":"I learned to play the ukulele"}],"accessibility',
 '"title":{"runs":[{"text":"I Became a Real Estate Agent"}],"accessibility',
 '"title":{"runs":[{"text":"The acting competition that went OFF the rails"}],"accessibility',
 '"title":{"runs":[{"text":"this will BLOW your mind"}],"accessibility',
 '"title":{"runs":[{"text":"The Worst Pickup Artist on Tik Tok"}],"accessibility',
 '"titl

In [42]:
# only the first 30 titles are shown
# this is because only the top of the page is loaded when visiting the url
# can automate scrolling using splinter
len(final_results)

30